# Hvordan fordele eiendeler med utgangspunkt i ønskelister?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import cvxopt

## 0-1 Integer Lineær programmering
"Minimize c*x subject to Ax <=b, x ∈ {0, 1}"

In [21]:
relation_weights = {
    'child': 1,
    'parent': 0.8,
    'sibling': 0.7,
    'grandchildren': 0.6,
    'grandparent': 0.4,
    'pibling': 0.15,
    'other': 0.1
}

relation_priority = ['child', 'parent', 'sibling', 'grandchildren', 'grandparent', 'pibling', 'other']

In [22]:
users = ['A', 'B', 'C']
relations = np.random.choice(relation_priority, len(users))
assets = ['a', 'b', 'c', 'd', 'e']

wishlists = [np.random.permutation(len(assets)) for u in users]
wish_hstack = np.hstack(wishlists) - len(assets)
c = np.array([wish_hstack[i] * relation_weights[relations[i//len(assets)]] for i in range(len(wish_hstack))])
c

array([-0.15, -0.6 , -0.45, -0.3 , -0.75, -2.1 , -1.4 , -0.7 , -2.8 ,
       -3.5 , -0.45, -0.15, -0.6 , -0.75, -0.3 ])

In [23]:
relations

array(['pibling', 'sibling', 'pibling'], dtype='<U13')

In [24]:
weight_factor = 1 / max(relation_weights[relation] for relation in relations)
weight_factor

1.4285714285714286

In [33]:
wishlists

[array([4, 1, 2, 3, 0]), array([2, 3, 4, 1, 0]), array([2, 4, 1, 0, 3])]

In [25]:
max_one_user_per_asset = [[1 if (i-j)%len(assets)==0 else 0 for i in range(len(c))] for j in range(len(assets))]
fair_distribution = [[wish_hstack[i] if len(assets)*j <= i < len(assets)*(j+1) else 0 for i in range(len(c))] for j in range(len(users))]
A = np.array(max_one_user_per_asset + fair_distribution)
A

array([[ 1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1],
       [-1, -4, -3, -2, -5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, -3, -2, -1, -4, -5,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -3, -1, -4, -5, -2]])

In [26]:
max_one_user_per_asset_sum = [1] * len(assets)
fair_distribution_sum = [wish_hstack[:len(assets)].sum() * relation_weights[relation] * weight_factor / len(users) for relation in relations]
b = np.array(max_one_user_per_asset_sum + fair_distribution_sum)
b

array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
       -1.07142857, -5.        , -1.07142857])

In [27]:
A.shape

(8, 15)

In [28]:
G = cvxopt.matrix(A, tc='d')
G

<8x15 matrix, tc='d'>

In [29]:
h=cvxopt.matrix(b, tc='d')
h

<8x1 matrix, tc='d'>

In [30]:
from cvxopt.glpk import ilp

solution = ilp(c=cvxopt.matrix(c, tc='d'),
               G=cvxopt.matrix(A, tc='d'),
               h=cvxopt.matrix(b, tc='d'),
               B=set(range(len(c))))
solution

('optimal', <15x1 matrix, tc='d'>)

In [31]:
opt = np.array(list(solution[1]))
opt

array([0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0.])

In [32]:
np.matmul(A, opt)

array([  1.,   1.,   1.,   1.,   1.,  -4., -12.,  -4.])

Litt testing og logikk tilsier at denne algoritmen er optimal og rettferdig. Vi går derfor for binær linear programmering som løsning.

## Testing av full_distribution()

In [2]:
import os, sys
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
sys.path.insert(0, os.path.abspath("../backend"))
os.environ["DJANGO_SETTINGS_MODULE"] = "backend.settings"

In [3]:
import django
django.setup()

In [4]:
from roddi.models import *

In [5]:
from roddi.test import setup_test_data
setup_test_data.main()

In [12]:
estate = Estate.objects.all().get()
users = User.objects.all()

In [7]:
estate.full_distribution()

In [55]:
[Relation.objects.all().get(user=u).relation for u in users]

['pibling', 'sibling', 'parent']

In [56]:
[(user, user.get_ordered_wishlist()) for user in users]

[(<User: Daniel>,
  [<Asset: Maleri>,
   <Asset: Gevær>,
   <Asset: Stol>,
   <Asset: PC>,
   <Asset: Vase>,
   <Asset: Bord>]),
 (<User: Philip>,
  [<Asset: Gevær>,
   <Asset: Stol>,
   <Asset: Vase>,
   <Asset: Bord>,
   <Asset: PC>,
   <Asset: Maleri>]),
 (<User: Steffen>,
  [<Asset: Stol>,
   <Asset: Gevær>,
   <Asset: Bord>,
   <Asset: Vase>,
   <Asset: Maleri>,
   <Asset: PC>])]

In [57]:
[(user, user.obtained_assets.all()) for user in users]

[(<User: Daniel>, <QuerySet [<Asset: Vase>]>),
 (<User: Philip>, <QuerySet [<Asset: Gevær>, <Asset: PC>]>),
 (<User: Steffen>, <QuerySet [<Asset: Maleri>, <Asset: Bord>, <Asset: Stol>]>)]

In [58]:
estate.is_complete

True

Alt ser ut til å fungere

In [14]:
?ilp

## Debugging

In [13]:
setup_test_data.main()

In [6]:
estate = Estate.objects.all().get()
users = User.objects.all()
assets = Asset.objects.all()

In [7]:
assets = [a for a in assets if a.to_be_distributed]
asset_ids = [a.id for a in assets]
relations = [Relation.objects.all().get(user=u, estate=estate).relation for u in users]
wishlists = [[a.id for a in u.get_ordered_wishlist() if a.id in asset_ids] for u in users]

In [8]:
asset_ids

[0, 1, 2, 3, 4, 5]

In [9]:
relations

['pibling', 'sibling', 'parent']

In [10]:
wishlists

[[4, 5, 0, 3, 1, 2], [2, 0, 5, 3, 4, 1], [4, 2, 1, 5, 0, 3]]

In [11]:
# List of numbers representing priorities. [[1, 0, 2], ...] means that the first user wants asset nr 2 the most, then asset nr 1, then asset nr 3
priorities = [[wishlist.index(asset_id) for asset_id in asset_ids] for wishlist in wishlists]

# One-dimensional list with negative priorities. indexed_wishlists = [[1, 0, 2], ...] would give wish_hstack = [-2, -3, -1, ...]
wish_hstack = np.hstack(priorities) - len(assets)

In [12]:
priorities

[[2, 4, 5, 3, 0, 1], [1, 5, 0, 3, 4, 2], [4, 2, 1, 5, 0, 3]]

In [13]:
wish_hstack

array([-4, -2, -1, -3, -6, -5, -5, -1, -6, -3, -2, -4, -2, -4, -5, -1, -6,
       -3])

In [14]:
# c defines the minimization function. We want to minimize the sum of products on the form negative_asset_priority * relation_weight for every asset-user-pair selected in the final matching
c = np.array([wish_hstack[i] * RELATION_WEIGHTS[relations[i//len(assets)]] for i in range(len(wish_hstack))])

In [15]:
c

array([-0.6 , -0.3 , -0.15, -0.45, -0.9 , -0.75, -3.5 , -0.7 , -4.2 ,
       -2.1 , -1.4 , -2.8 , -1.6 , -3.2 , -4.  , -0.8 , -4.8 , -2.4 ])

In [16]:
max_one_user_per_asset = [[1 if (i-j) % len(assets) == 0 else 0 for i in range(len(c))] for j in range(len(assets))]
max_one_user_per_asset_sum = [1] * len(assets)

In [17]:
max_one_user_per_asset

[[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]]

In [18]:
# We measure the 'satisfaction' a pairing gives to a user as the sum of negative priorities for each asset he/she gets in the distribution. Smaller is better
# Here, we define a rule saying each user deserves a satisfaction less than the sum of their negative priorities divided by the number of users.
# We also multiply with a weight_factor, so that uncles and grandparents deserves less 'satisfaction' than children and siblings, etc.
fair_distribution = [[wish_hstack[i] if len(assets) * j <= i < len(assets) * (j+1) else 0 for i in range(len(wish_hstack))] for j in range(len(users))]
weight_factor = 1 / max(RELATION_WEIGHTS[relation] for relation in relations)
fair_distribution_sum = [wish_hstack[:len(assets)].sum() * RELATION_WEIGHTS[relation] * weight_factor / len(users) for relation in relations]

In [19]:
np.array(fair_distribution)

array([[-4, -2, -1, -3, -6, -5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0, -5, -1, -6, -3, -2, -4,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -2, -4, -5, -1,
        -6, -3]])

In [20]:
weight_factor

1.25

In [21]:
fair_distribution_sum

[-1.3125, -6.125, -7.0]

In [22]:
# Concatenate the requirements to arrays G and h
G = np.array(max_one_user_per_asset + fair_distribution)
h = np.array(max_one_user_per_asset_sum + fair_distribution_sum)

In [23]:
G

array([[ 1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,
         0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,
         0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,
         0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,
         0,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         1,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,
         0,  1],
       [-4, -2, -1, -3, -6, -5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0, -5, -1, -6, -3, -2, -4,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -2, -4, -5, -1,
        -6, -3]])

In [24]:
h

array([ 1.    ,  1.    ,  1.    ,  1.    ,  1.    ,  1.    , -1.3125,
       -6.125 , -7.    ])

In [25]:
solution = ilp(c=cvxopt.matrix(c, tc='d'),
                             G=cvxopt.matrix(G, tc='d'),
                             h=cvxopt.matrix(h, tc='d'),
                             B=set(range(len(c))))
opt = np.array(list(solution[1]))

In [26]:
opt

array([0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1.,
       0.])

In [27]:
users

<QuerySet [<User: Daniel>, <User: Philip>, <User: Steffen>]>

In [28]:
for i in range(len(opt)):
    if opt[i] == 1:
        asset_index = i % len(assets)
        user_index = i // len(assets)
        users[user_index].obtained_assets.add(assets[asset_index])
        assets[asset_index].belongs_to = users[user_index]
        assets[asset_index].process()
        assets[asset_index].to_be_distributed = False
        assets[asset_index].save()

In [29]:
priorities

[[2, 4, 5, 3, 0, 1], [1, 5, 0, 3, 4, 2], [4, 2, 1, 5, 0, 3]]

In [30]:
assets

[<Asset: Vase>,
 <Asset: Maleri>,
 <Asset: Gevær>,
 <Asset: Bord>,
 <Asset: Stol>,
 <Asset: PC>]

In [31]:
[(user, user.get_ordered_wishlist()) for user in users]

[(<User: Daniel>,
  [<Asset: Stol>,
   <Asset: PC>,
   <Asset: Vase>,
   <Asset: Bord>,
   <Asset: Maleri>,
   <Asset: Gevær>]),
 (<User: Philip>,
  [<Asset: Gevær>,
   <Asset: Vase>,
   <Asset: PC>,
   <Asset: Bord>,
   <Asset: Stol>,
   <Asset: Maleri>]),
 (<User: Steffen>,
  [<Asset: Stol>,
   <Asset: Gevær>,
   <Asset: Maleri>,
   <Asset: PC>,
   <Asset: Vase>,
   <Asset: Bord>])]

In [32]:
[(user, user.obtained_assets.all()) for user in users]

[(<User: Daniel>, <QuerySet [<Asset: Bord>]>),
 (<User: Philip>, <QuerySet [<Asset: Vase>, <Asset: Gevær>, <Asset: PC>]>),
 (<User: Steffen>, <QuerySet [<Asset: Maleri>, <Asset: Stol>]>)]